In [1]:
import datetime
from dateutil.relativedelta import relativedelta
#from datetime import timedelta
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.ticker as mtick
import matplotlib.gridspec as gridspec
import seaborn as sns

# import plotly.plotly as py
# import plotly.graph_objs as go

In [2]:
dict_dtypes={'BusinessDate': object, 'LocationNo': str, 'TestGroup': int}
df_BuyHeaderMetrics = pd.read_csv('./BuyHeaderMetrics.csv', dtype=dict_dtypes, header=0, index_col=0)
df_BuyItemMetrics = pd.read_csv('./BuyItemMetrics.csv', dtype=dict_dtypes, header=0, index_col=0)

ValueError: cannot safely convert passed user dtype of int32 for float64 dtyped data in column 4

In [ ]:
df_BuyHeaderMetrics.info()

In [ ]:
StartDate = '2019-01-01'
filt_DropBeforeStartDate = df_BuyHeaderMetrics['BusinessDate'] < StartDate
df_BuyHeaderMetrics = df_BuyHeaderMetrics.drop(df_BuyHeaderMetrics.index[filt_DropBeforeStartDate], axis=0).reset_index(drop=True)

In [ ]:
df_BuyHeaderMetrics['total_BuyWait_Min'] = df_BuyHeaderMetrics['total_BuyWait_Sec']/60
df_BuyHeaderMetrics['rollavg_BuyWait_Min'] = df_BuyHeaderMetrics['rollavg_BuyWait_Sec']/60
df_BuyHeaderMetrics['diff_avg_BuyWait_Min'] = df_BuyHeaderMetrics['diff_avg_BuyWait_Sec']/60

In [ ]:
NoCalcColumns = ['BusinessDate', 'LocationNo', 'VersionNo', 'TestGroup', 'BuyType']
CalcColumns = [c for c in df_BuyHeaderMetrics.columns if c not in NoCalcColumns]

In [ ]:
def get_SummaryStatsAroundDate(df, date, LocName, CalcColumns):
    SumAroundDate = date
    filt_SelectGroup = df['LocationNo'] == LocName
    filt_SumAfterDate = (df['BusinessDate'] >= SumAroundDate) & filt_SelectGroup
    filt_SumBeforeDate = (df['BusinessDate'] < SumAroundDate) & filt_SelectGroup
    series_BeforeDate = df[filt_SumBeforeDate][CalcColumns].sum()
    series_AfterDate = df[filt_SumAfterDate][CalcColumns].sum()
    return series_BeforeDate, series_AfterDate

def get_EachStoreSummaryStatsAfterDate(df, date, list_locs):
    SumAfterDate = date
    filt_SelectGroup = df['LocationNo'].isin(list_locs)
    filt_SumAfterDate = (df['BusinessDate'] >= date) & filt_SelectGroup
    df_AfterDate = df[filt_SumAfterDate].groupby('LocationNo').sum()
    return df_AfterDate

In [ ]:
series_AllBeforeDate, series_AllAfterDate = get_SummaryStatsAroundDate(df_BuyHeaderMetrics, '2019-06-25', 'v1.r3', CalcColumns)
series_TG1BeforeDate, series_TG1AfterDate = get_SummaryStatsAroundDate(df_BuyHeaderMetrics, '2019-06-25', 'tg1', CalcColumns)
series_TG2BeforeDate, series_TG2AfterDate = get_SummaryStatsAroundDate(df_BuyHeaderMetrics, '2019-06-25', 'tg2', CalcColumns)

In [ ]:
list_CompChainLoc_All = np.array([series_AllAfterDate['count_SuggestedOfferChain'], 
                     series_AllAfterDate['count_SuggestedOfferLoc']])
list_CompChainLocAdj_All = np.array([series_AllAfterDate['count_SuggestedOfferChainAdj'], 
                     series_AllAfterDate['count_SuggestedOfferLocAdj']])
list_CompChainLocNestedAdj_All = np.array([[list_CompChainLoc_All[0]-list_CompChainLocAdj_All[0], 
                                            list_CompChainLocAdj_All[0]],
                                            [list_CompChainLoc_All[1]-list_CompChainLocAdj_All[1], 
                                            list_CompChainLocAdj_All[1]]])

list_CompChainLoc_TG1 = np.array([series_TG1AfterDate['count_SuggestedOfferChain'], 
                     series_TG1AfterDate['count_SuggestedOfferLoc']])
list_CompChainLocAdj_TG1 = np.array([series_TG1AfterDate['count_SuggestedOfferChainAdj'], 
                     series_TG1AfterDate['count_SuggestedOfferLocAdj']])
list_CompChainLocNestedAdj_TG1 = np.array([[list_CompChainLoc_TG1[0]-list_CompChainLocAdj_TG1[0], 
                                            list_CompChainLocAdj_TG1[0]],
                                            [list_CompChainLoc_TG1[1]-list_CompChainLocAdj_TG1[1], 
                                            list_CompChainLocAdj_TG1[1]]])

list_CompChainLoc_TG2 = np.array([series_TG2AfterDate['count_SuggestedOfferChain'], 
                     series_TG2AfterDate['count_SuggestedOfferLoc']])
list_CompChainLocAdj_TG2 = np.array([series_TG2AfterDate['count_SuggestedOfferChainAdj'], 
                     series_TG2AfterDate['count_SuggestedOfferLocAdj']])
list_CompChainLocNestedAdj_TG2 = np.array([[list_CompChainLoc_TG2[0]-list_CompChainLocAdj_TG2[0], 
                                            list_CompChainLocAdj_TG2[0]],
                                            [list_CompChainLoc_TG2[1]-list_CompChainLocAdj_TG2[1], 
                                            list_CompChainLocAdj_TG2[1]]])

In [ ]:
fig = plt.figure(figsize=(12,10))



ax1 = plt.subplot2grid((5, 4), (0, 0),  rowspan=3, colspan=4)
ax2 = plt.subplot2grid((5, 4), (3, 0),  rowspan=2, colspan=2)
ax3 = plt.subplot2grid((5, 4), (3, 2),  rowspan=2, colspan=2)
fig.subplots_adjust(top=.99, bottom=0, hspace=0, wspace=0)

PieWidth = 0.3
list_OuterPieLabels = ['% Offers Chain-Based', '% Offers Location-Based']
list_InnerPieLabels = ['% Offers Chain-Based, Accepted by Buyer', '% Offers Chain-Based, Changed by Buyer', 
                       '% Offers Location-Based, Accepted by Buyer', '% Offers Location-Based, Changed by Buyer']

cmap = plt.get_cmap("tab20c")
outer_colors = cmap(np.arange(3)*4)
inner_colors = cmap(np.array([1, 2, 5, 6, 9, 10]))


ax1.pie(list_CompChainLoc_All, labels=None, autopct='%1.1f%%', 
       radius=1, startangle=90, pctdistance=0.85, colors=outer_colors,
       wedgeprops=dict(width=PieWidth, edgecolor='w'), textprops=dict(color='w', fontsize=12, fontweight='bold'))


ax1.pie(list_CompChainLocNestedAdj_All.flatten(), labels=None, 
       autopct='%1.1f%%', radius=1-PieWidth, startangle=90, pctdistance=0.8, colors=inner_colors,
       wedgeprops=dict(width=PieWidth, edgecolor='w'), 
       textprops=dict(color='w', fontsize=10, fontweight='bold', rotation=0))

ax1.legend(labels=list_OuterPieLabels + list_InnerPieLabels, loc="lower center",
           bbox_to_anchor=(1, 0.1, 0.5, 1))


ax2.pie(list_CompChainLoc_TG1, labels=None, autopct='%1.1f%%', 
       radius=1, startangle=90, pctdistance=0.85, colors=outer_colors,
       wedgeprops=dict(width=PieWidth, edgecolor='w'), textprops=dict(color='w', fontsize=12, fontweight='bold'))


ax2.pie(list_CompChainLocNestedAdj_TG1.flatten(), labels=None, 
       autopct='%1.1f%%', radius=1-PieWidth, startangle=90, pctdistance=0.8, colors=inner_colors,
       wedgeprops=dict(width=PieWidth, edgecolor='w'), 
       textprops=dict(color='w', fontsize=10, fontweight='bold', rotation=0))
ax2.set_title('Test Group 1')



ax3.pie(list_CompChainLoc_TG2, labels=None, autopct='%1.1f%%', 
       radius=1, startangle=90, pctdistance=0.85, colors=outer_colors,
       wedgeprops=dict(width=PieWidth, edgecolor='w'), textprops=dict(color='w', fontsize=12, fontweight='bold'))


ax3.pie(list_CompChainLocNestedAdj_TG2.flatten(), labels=None, 
       autopct='%1.1f%%', radius=1-PieWidth, startangle=90, pctdistance=0.8, colors=inner_colors,
       wedgeprops=dict(width=PieWidth, edgecolor='w'), 
       textprops=dict(color='w', fontsize=10, fontweight='bold', rotation=0))
ax3.set_title('Test Group 2')

plt.suptitle('Suggested Offers Release 3 - Chain-based vs Location-based Offers', fontsize=14)

plt.show()

In [ ]:
series_LocBeforeDate, series_LocAfterDate = get_SummaryStatsAroundDate(df_BuyHeaderMetrics, '2019-06-25', '00097', CalcColumns)

list_CompChainLoc_Loc = np.array([series_LocAfterDate['count_SuggestedOfferChain'], 
                     series_LocAfterDate['count_SuggestedOfferLoc']])
list_CompChainLocAdj_Loc = np.array([series_LocAfterDate['count_SuggestedOfferChainAdj'], 
                     series_LocAfterDate['count_SuggestedOfferLocAdj']])
list_CompChainLocNestedAdj_Loc = np.array([[list_CompChainLoc_Loc[0]-list_CompChainLocAdj_Loc[0], 
                                            list_CompChainLocAdj_Loc[0]],
                                            [list_CompChainLoc_Loc[1]-list_CompChainLocAdj_Loc[1], 
                                            list_CompChainLocAdj_Loc[1]]])

In [ ]:
PieWidth = 0.3
fig, ax = plt.subplots(figsize=(8,8))
list_OuterPieLabels = ['% Offers Chain-Based', '% Offers Location-Based']
list_InnerPieLabels = ['% Offers Chain-Based, Accepted by Buyer', '% Offers Chain-Based, Changed by Buyer', 
                       '% Offers Location-Based, Accepted by Buyer', '% Offers Location-Based, Changed by Buyer']

cmap = plt.get_cmap("tab20c")
outer_colors = cmap(np.arange(3)*4)
inner_colors = cmap(np.array([1, 2, 5, 6, 9, 10]))


ax.pie(list_CompChainLoc_Loc, labels=None, autopct='%1.1f%%', 
       radius=1, startangle=90, pctdistance=0.85, colors=outer_colors,
       wedgeprops=dict(width=PieWidth, edgecolor='w'), textprops=dict(color='w', fontsize=12, fontweight='bold'))


ax.pie(list_CompChainLocNestedAdj_Loc.flatten(), labels=None, 
       autopct='%1.1f%%', radius=1-PieWidth, startangle=90, pctdistance=0.8, colors=inner_colors,
       wedgeprops=dict(width=PieWidth, edgecolor='w'), 
       textprops=dict(color='w', fontsize=10, fontweight='bold', rotation=0))
plt.legend(labels=list_OuterPieLabels + list_InnerPieLabels, loc="center left",
           bbox_to_anchor=(1, 0, 0.5, 1))

plt.title('Suggested Offers Release 3 - Chain-based vs Location-based Offers', fontsize=14)

plt.show()

In [ ]:
filt_V1R3Locs = df_BuyHeaderMetrics['VersionNo'] == 'v1.r3'
list_R3Locs = list(df_BuyHeaderMetrics[filt_V1R3Locs]['LocationNo'].unique())

In [ ]:
list_R3Locs = [i for i in list_R3Locs if i not in ['tg1', 'tg2']]

In [ ]:
df_PctLocOffers = get_EachStoreSummaryStatsAfterDate(df_BuyHeaderMetrics, '2019-06-25', list_R3Locs).reset_index()
df_PctLocOffers['pct_LocOffers'] = df_PctLocOffers['count_SuggestedOfferLoc']/df_PctLocOffers['total_SuggestedOfferQuantity']

In [ ]:
sns.set_style('whitegrid')
list_PlotOrder = df_PctLocOffers.sort_values('pct_LocOffers')['LocationNo'].to_list()

fig, ax = plt.subplots(figsize=(8,6))
sns.barplot(x='LocationNo', y='pct_LocOffers', data=df_PctLocOffers, ax=ax, 
            order=list_PlotOrder, palette = 'cividis', alpha = 0.85)
#ax.set_axisbelow(False)
#ax.grid(axis='y', color='white')
plt.xticks(rotation=90)
ax.set_xlabel('Store #', fontsize=12)
ax.set_ylabel('% Offers Location-Based', fontsize=12)
ax.yaxis.set_major_formatter(mtick.StrMethodFormatter('{x:,.1%}')) 
plt.title('Percent Location-Based Offers By Store', fontsize=14)
plt.show()

In [ ]:
df_PctLocOffers.sort_values('pct_LocOffers')['LocationNo'].to_list()